## NOMS Paper

In [ ]:
using CSV
using DataFrames
using Distributions
using Glob
using HMMBase
using JSON
using ParsimoniousMonitoring: LogEntry
using Statistics

In [ ]:
parse_log(d::Dict) = LogEntry(map(x -> d[string(x)], fieldnames(LogEntry))...)

function load_results(filename)
    obj = JSON.parsefile(filename)
    Dict(k => map(parse_log, v) for (k, v) in obj)
end

# TODO: Move to dedicated file
read_ts(filename) = CSV.read(filename, header = ["timestep", "rtt"])

function load_scenario(path)
    files = map(x -> splitext(x)[1], glob("*.csv", path))
    models = []
    series = []
    for file in files
        push!(models, read_model("$file.json"))
        push!(series, read_ts("$file.csv"))
    end
    data = hcat(map(x -> x.rtt, series)...)
    name = splitpath(path)[end]
    name, models, data
end

function read_model(filename)
    obj = JSON.parsefile(filename)
    A = permutedims(hcat(obj["transmat"]...))
    B = map(obj["states"]) do (_, d)
        components = map(zip(d["means"], d["variances"])) do (μ, σ2)
            Normal(μ, sqrt(σ2))
        end
        MixtureModel(components, [d["weights"]...])
    end
    HMM(A, B)
end

### 5.2 Validation against real latency data

In [ ]:
scenarios = [
    "../data/noms_scenarios/Haifa_Santiago/",
    "../data/noms_scenarios/Paris_Santiago/",
    "../data/noms_scenarios/Paris_Tokyo/",
    "../data/noms_scenarios/Singapore_HongKong/"
];

In [ ]:
τmax = 100
for file in scenarios
    models = load_scenario(file)[2]
    @show file
    @show prod(m -> size(m, 1) * τmax, models)
end

In [ ]:
168000000 / 10^6

In [ ]:
files = [
    "../results/Haifa_Santiago.json",
    "../results/Paris_Santiago.json",
    "../results/Paris_Tokyo.json",
    "../results/Singapore_HongKong.json"
]

function min_delay_gap(log)
    delay = sum(skipmissing(map(x -> isnothing(x.delay) ? missing : x.delay, log)))
    delay_opt = sum(skipmissing(map(x -> isnothing(x.delay_opt) ? missing : x.delay_opt, log)))
    (delay - delay_opt) / delay_opt
end

metrics = Dict(
    "processing_time" => log -> mean(x -> x.time, log),
    "avg_measures" => log -> mean(x -> sum(x.a), log),
    "min_delay_reached" => log -> mean(x -> x.delay == x.delay_opt, log) * 100,
    "min_delay_gap" => log -> min_delay_gap(log) * 100
);

In [ ]:
df = DataFrame([String[], String[], String[], Real[]], [:scenario, :policy, :metric, :value])
for file in files
    scenario = splitext(basename(file))[1]
    results = load_results(file)
    for (policy, log) in results, (metric, f) in metrics
        push!(df, (scenario, policy, metric, f(log)))
    end
end
df = unstack(df, :metric, :value);

In [ ]:
df[df.scenario .== "Singapore_HongKong",:]